In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# URL de base pour la catégorie Maison
BASE_URL = "https://www.expat-dakar.com/maison"

# Headers pour simuler un navigateur
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Fonction pour récupérer les liens des annonces sur une page donnée
def get_announcement_links(base_url, page):
    response = requests.get(f"{base_url}?page={page}", headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Extraire tous les liens des annonces
    links = []
    for item in soup.find_all("div", class_="listings-cards__list-item"):
        link_tag = item.find("a", href=True)
        if link_tag:
            href = link_tag['href']
            # Vérifiez que le lien est bien formaté
            if href.startswith("/"):  # Lien relatif
                links.append("https://www.expat-dakar.com" + href)
            elif href.startswith("http"):  # Lien absolu déjà formé
                links.append(href)
    
    return links

# Fonction pour récupérer les détails d'une annonce
def get_announcement_details(url):
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Récupérer les informations spécifiques
    details = {}
    try:
        # Titre et prix
        details["Titre"] = soup.find("h1", class_="listing-item__header").text.strip() if soup.find("h1", class_="listing-item__header") else "N/A"
        details["Prix"] = soup.find("span", class_="listing-card__price__value").text.strip() if soup.find("span", class_="listing-card__price__value") else "N/A"
        
        # Adresse
        location = soup.find("span", class_="listing-item__address-location")
        region = soup.find("span", class_="listing-item__address-region")
        details["Adresse"] = f"{location.text.strip()} {region.text.strip()}" if location and region else "N/A"
        
        # Récupérer les détails listés dans "Détails"
        for dt, dd in zip(
            soup.find_all("dt", class_="listing-item__properties__title"),
            soup.find_all("dd", class_="listing-item__properties__description")
        ):
            details[dt.text.strip()] = dd.text.strip()
        
        # Description
        details["Description"] = soup.find("div", class_="listing-item__description").text.strip() if soup.find("div", class_="listing-item__description") else "N/A"
    except Exception as e:
        print(f"Erreur lors de l'extraction de {url}: {e}")
    
    return details

# Récupérer tous les liens et les détails des annonces sur plusieurs pages
print("Début du scraping des annonces de la catégorie Maison...")
all_links = []
data = []  # Pour stocker les détails des annonces

# Boucle pour parcourir les pages 
MAX_PAGES = 600 
for page in range(1, MAX_PAGES + 1):
    print(f"Scraping de la page {page}...")
    links = get_announcement_links(BASE_URL, page)
    all_links.extend(links)
    print(f"{len(links)} liens trouvés sur la page {page}.")
    
    # Récupérer les détails pour chaque annonce de cette page
    for link in links:
        print(f"Scraping des détails pour : {link}")
        details = get_announcement_details(link)
        data.append(details)
        time.sleep(1)  # Pause pour éviter de surcharger le serveur

# Sauvegarder les résultats dans un fichier Excel
df = pd.DataFrame(data)
df.to_excel("annonces_maison_expat_dakar.xlsx", index=False)
print("Les données ont été sauvegardées dans 'annonces_maison_expat_dakar.xlsx'.")


Début du scraping des annonces de la catégorie Maison...
Scraping de la page 1...
10 liens trouvés sur la page 1.
Scraping des détails pour : https://www.expat-dakar.com/annonce/combine-beko-3-tiroirs-6219835
Scraping des détails pour : https://www.expat-dakar.com/annonce/salon-7-place-6242107
Scraping des détails pour : https://www.expat-dakar.com/annonce/salon-7-place-6242106
Scraping des détails pour : https://www.expat-dakar.com/annonce/chambre-a-coucher-6242105
Scraping des détails pour : https://www.expat-dakar.com/annonce/lots-de-6-chaises-style-scandinave-malmo-4867770
Scraping des détails pour : https://www.expat-dakar.com/annonce/lots-de-4-tabourets-style-scandinave-malmo-4867788
Scraping des détails pour : https://www.expat-dakar.com/annonce/salon-7places-6238042
Scraping des détails pour : https://www.expat-dakar.com/annonce/salon-7-place-6241216
Scraping des détails pour : https://www.expat-dakar.com/annonce/extracteur-de-jus-1000-watts-6228618
Scraping des détails pour : 